In [ ]:
import pandas as pd
from botasaurus import *
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import xml.etree.ElementTree as ET
from config.config_jupyter import set_wd
set_wd()

In [35]:
@browser(headless=True, output=None, profile=None)
def scrape_links_task(driver: AntiDetectDriver, data):
    driver.get("https://www.indec.gob.ar/indec/web/Nivel4-Tema-3-35-45")
    # xls_div = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'a-color2')))
    xlsx_div = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'a-color2'))
    )
    hrefs=[]
    for element in xlsx_div:
        elemento=element.get_attribute('href')
        if elemento != None:
            if (elemento.endswith("xls") | elemento.endswith("xml")):
                hrefs.append(elemento)
    # links = driver.get_links()
    return {
        "links": hrefs
    }

In [36]:
links = scrape_links_task()
links

In [16]:
sdmx_code

'Q.N.AR.W1.S1.S1.T.B.CA._Z._Z._Z.USD._T._X.N'

In [ ]:
url = "https://www.indec.gob.ar/ftp/cuadros/economia/BOP.xml"
response = requests.get(url)
if response.status_code == 200:
    header, dataset = ET.fromstring(response.content)
    for child in dataset:
        print('.'.join(child.attrib.values()))
    # You can also access specific elements by tag name:
    # for element in root.findall('your_tag_name'):
    #     print(element.tag, element.text)
else:
    print("Failed to retrieve XML data")



In [5]:
import requests
import xml.etree.ElementTree as ET

# URL of the XML file
url = "https://www.indec.gob.ar/ftp/cuadros/economia/BOP.xml"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the XML content
    header, dataset = ET.fromstring(response.content)

    # Define the SDMX code you want to search for
    sdmx_code = "Q.N.AR.W1.S1.S1.T.B.CA._Z._Z._Z.USD._T._X.N"

    # Iterate through the Series elements
    for series in dataset.findall('.//Series'):
        # Construct the series ID using attributes
        series_id = ".".join([series.get(attr) for attr in ['FREQ', 'REF_AREA', 'COUNTERPART_AREA',
                                                            'REF_SECTOR', 'COUNTERPART_SECTOR',
                                                            'FLOW_STOCK_ENTRY', 'ACCOUNTING_ENTRY',
                                                            'INT_ACC_ITEM', 'FUNCTIONAL_CAT', 'INSTR_ASSET',
                                                            'MATURITY', 'UNIT_MEASURE', 'CURRENCY_DENOM',
                                                            'VALUATION', 'COMP_METHOD', 'TIME_FORMAT']])
        
        # If the series ID matches the desired SDMX code, print it
        if series_id == sdmx_code:
            print("Found the series with SDMX code:", sdmx_code)
            for obs in series.findall('Obs'):
                print("Time Period:", obs.get('TIME_PERIOD'), "Value:", obs.get('OBS_VALUE'))
            break
    else:
        print("Series with SDMX code", sdmx_code, "not found.")
else:
    print("Failed to retrieve XML data")


Series with SDMX code Q.N.AR.W1.S1.S1.T.B.CA._Z._Z._Z.USD._T._X.N not found.
